Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `events`. Open the `4.0 events-gen-kafka.ipynb` notebook and execute the cell. This notebook produces an event record every second and put it on a Kafka topic called `events`. 

In [1]:
from delta import  # Delta Lake 라이브러리 임포트 configure_spark_with_delta_pip
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, from_json, window, count, to_timestamp
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, IntegerType, StringType

In [2]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("handle-late-and-out-of-order-data")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f077b092-f976-457f-be51-e433b924d7bd;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in centra

In [3]:
df = (spark.readStream  # 스트리밍 데이터 읽기
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "events")
      .option("startingOffsets", "latest")
      .load(  # 파일 로드))

In [4]:
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드('user_id', IntegerType(), True),
    StructField  # 구조체 필드('event_type', StringType(), True),
    StructField  # 구조체 필드('event_time', StringType(), True),
    StructField  # 구조체 필드('processing_time', StringType(), True)])

df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정'value', from_json(col('value').cast("STRING"), schema))

In [5]:
df = (df.select(  # 컬럼 선택
          col(  # 컬럼 참조'value.user_id').alias('user_id'),
          col(  # 컬럼 참조'value.event_type').alias('event_type'),
          col(  # 컬럼 참조'value.event_time').alias('event_time'),
          col(  # 컬럼 참조'value.processing_time').alias('processing_time'))
      .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"event_time", to_timestamp(col("event_time"), "MM/dd/yyyy, HH:mm:ss" ))
      .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"processing_time", to_timestamp(col("processing_time"), "MM/dd/yyyy, HH:mm:ss"))     )

In [6]:
# Define the watermark logic for the streaming DataFrame
df = df.withWatermark("event_time", "10 seconds")

In [7]:
df = (df
      .groupBy(  # 그룹화window(col("event_time"), "1 minute", "1 minute"), col("user_id"))
      .count()  # 행 개수 계산.alias("NumberOfEvents"))

In [8]:
query = (df.writeStream  # 스트리밍 데이터 쓰기
    .outputMode(  # 스트리밍 출력 모드 설정'update')
    .format('console')
    .option("truncate", False)
    .start()  # 스트리밍 시작)

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|user_id|count|
+------+-------+-----+
+------+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |user_id|count|
+------------------------------------------+-------+-----+
|{2024-02-04 18:09:00, 2024-02-04 18:10:00}|42     |1    |
|{2024-02-04 18:09:00, 2024-02-04 18:10:00}|64     |1    |
|{2024-02-04 18:10:00, 2024-02-04 18:11:00}|59     |1    |
+------------------------------------------+-------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |user_id|count|
+------------------------------------------+-------+-----+
|{2024-02-04 18:10:00, 2024-02-04 18:11:00}|81     |1    |
+------------------------------------------+-------+-----+



In [9]:
query.stop()

24/02/04 18:10:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 3, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
24/02/04 18:10:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 3, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.


In [10]:
spark.stop()  # Spark 세션 종료 - 리소스 정리 